In [28]:
import firebase_admin
from firebase_admin import credentials, db
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import datetime

# ================= CONFIGURASI =================
# 1. Lokasi & Waktu
LAT = -7.66  # Kebumen
LON = 109.65
# Format: YYYY-MM-DD
TARGET_DATE = "2025-11-29" 
START_TIME_WIB = "03:09:00"
END_TIME_WIB   = "07:46:00"
FORMAT_KEY = 'UNIX'  # <--- GANTI INI SESUAI DATABASE MU! (Saran: Coba UNIX dulu)

In [34]:
# 2. Firebase Config
# Ganti dengan path file JSON Anda
CRED_PATH = "/work/staklimjerukagung-firebase-admin.json" 
DB_URL = "https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/"
# Path di database tempat data sensor disimpan
# Misal: sensors/kbm-05/logs
DB_NODE_PATH = "/auto_weather_stat/id-05/data" 

# ================= 1. SETUP KONEKSI =================
# Setup Firebase
if not firebase_admin._apps:
    cred = credentials.Certificate(CRED_PATH)
    firebase_admin.initialize_app(cred, {'databaseURL': DB_URL})

# Setup Open-Meteo
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
client = openmeteo_requests.Client(session=retry_session)

In [37]:
def get_data_interpolated():
    # Gunakan endpoint Forecast jika tanggal masa depan, Archive jika masa lalu
    # Untuk 2025 (masa depan), kita coba forecast, tapi karena jauh bgt mungkin error.
    # Disini saya pakai archive sebagai contoh struktur data.
    url = "https://archive-api.open-meteo.com/v1/archive" 
    
    params = {
        "latitude": LAT, "longitude": LON,
        "start_date": TARGET_DATE, "end_date": TARGET_DATE,
        "hourly": ["temperature_2m", "relative_humidity_2m"],
        "timezone": "Asia/Jakarta"
    }
    
    print("🚀 Mengambil data...")
    try:
        responses = client.weather_api(url, params=params)
        response = responses[0]
    except Exception as e:
        print(f"❌ Error API: {e}")
        return None

    # Proses Data
    hourly = response.Hourly()
    df = pd.DataFrame({
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ),
        "temperature": hourly.Variables(0).ValuesAsNumpy(),
        "humidity": hourly.Variables(1).ValuesAsNumpy()
    })
    
    # 1. Konversi ke WIB
    df = df.set_index('date')
    df.index = df.index.tz_convert('Asia/Jakarta')
    
    # 2. Resample ke Menit & Interpolasi
    df = df.resample('1T').asfreq()
    df = df.interpolate(method='linear')
    
    # 3. Potong Waktu
    start = f"{TARGET_DATE} {START_TIME_WIB}"
    end   = f"{TARGET_DATE} {END_TIME_WIB}"
    df = df.loc[start:end]
    
    # 4. Hapus data kosong (Penting biar gak Invalid Date/NaN)
    df = df.dropna()
    
    return df

def upload_fix(df):
    ref = db.reference(DB_NODE_PATH)
    print(f"🔥 Uploading dengan Format Key: {FORMAT_KEY}")
    
    for index, row in df.iterrows():
        # --- LOGIKA FORMAT KEY ---
        if FORMAT_KEY == 'UNIX':
            # Mengubah ke Angka Detik (Timestamp)
            key = str(int(index.timestamp()))
        elif FORMAT_KEY == 'ISO':
            # Format: 2025-11-29T03:09:00
            key = index.strftime('%Y-%m-%dT%H:%M:%S')
        else:
            # Format: 2025-11-29 03:09:00
            key = index.strftime('%Y-%m-%d %H:%M:%S')
            
        data = {
            'temperature': round(float(row['temperature']), 2),
            'humidity': round(float(row['humidity']), 2),
            'replaced': True # Penanda
        }
        
        # Cek print dulu sebelum upload beneran
        # print(f"Key: {key} | Data: {data}") 
        
        try:
            ref.child(key).update(data)
            print(f"✅ Updated: {key}")
        except Exception as e:
            print(f"❌ Gagal: {e}")

# EKSEKUSI
if __name__ == "__main__":
    df_hasil = get_data_interpolated()
    if df_hasil is not None and not df_hasil.empty:
        # Tampilkan sampel data dan Key yang akan terbentuk
        print("\n--- CEK DULU SEBELUM UPLOAD ---")
        sampel_waktu = df_hasil.index[0]
        
        test_key_unix = str(int(sampel_waktu.timestamp()))
        test_key_iso = sampel_waktu.strftime('%Y-%m-%dT%H:%M:%S')
        test_key_simple = sampel_waktu.strftime('%Y-%m-%d %H:%M:%S')
        
        print(f"Contoh Data Asli (Pandas): {sampel_waktu}")
        print(f"Opsi UNIX   (Angka) : {test_key_unix}")
        print(f"Opsi ISO    (Huruf T): {test_key_iso}")
        print(f"Opsi SIMPLE (Spasi)  : {test_key_simple}")
        print(f"\nKamu sedang memilih format: {FORMAT_KEY}")
        
        x = input("Apakah format key sudah sesuai dengan database lama? (y/n): ")
        if x.lower() == 'y':
            upload_fix(df_hasil)
        else:
            print("Dibatalkan. Silakan ganti 'FORMAT_KEY' di dalam kode.")
    else:
        print("Data kosong atau gagal diambil.")

🚀 Mengambil data...

--- CEK DULU SEBELUM UPLOAD ---
Contoh Data Asli (Pandas): 2025-11-29 03:09:00+07:00
Opsi UNIX   (Angka) : 1764360540
Opsi ISO    (Huruf T): 2025-11-29T03:09:00
Opsi SIMPLE (Spasi)  : 2025-11-29 03:09:00

Kamu sedang memilih format: UNIX
🔥 Uploading dengan Format Key: UNIX
✅ Updated: 1764360540
✅ Updated: 1764360600
✅ Updated: 1764360660
✅ Updated: 1764360720
✅ Updated: 1764360780
✅ Updated: 1764360840
✅ Updated: 1764360900
✅ Updated: 1764360960
✅ Updated: 1764361020
✅ Updated: 1764361080
✅ Updated: 1764361140
✅ Updated: 1764361200
✅ Updated: 1764361260
✅ Updated: 1764361320
✅ Updated: 1764361380
✅ Updated: 1764361440
✅ Updated: 1764361500
✅ Updated: 1764361560
✅ Updated: 1764361620
✅ Updated: 1764361680
✅ Updated: 1764361740
✅ Updated: 1764361800
✅ Updated: 1764361860
✅ Updated: 1764361920
✅ Updated: 1764361980
✅ Updated: 1764362040
✅ Updated: 1764362100
✅ Updated: 1764362160
✅ Updated: 1764362220
✅ Updated: 1764362280
✅ Updated: 1764362340
✅ Updated: 1764362400
✅

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>